# CNNs for Image Classification

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import os
import json
from datetime import datetime
from PIL import Image

import tensorflow as tf
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D

SEED = 1234
tf.random.set_seed(1234)

cwd = os.getcwd()

## Datasets loading

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

bs = 8
img_w = 256
img_h = 256
validation_split = 0.1

classes = [
    'owl',              # 0
    'galaxy',           # 1
    'lightning',        # 2
    'wine-bottle',      # 3
    't-shirt',          # 4
    'waterfall',        # 5
    'sword',            # 6
    'school-bus',       # 7
    'calculator',       # 8
    'sheet-music',      # 9
    'airplanes',        # 10
    'lightbulb',        # 11
    'skyscraper',       # 12
    'mountain-bike',    # 13
    'fireworks',        # 14
    'computer-monitor', # 15
    'bear',             # 16
    'grand-piano',      # 17
    'kangaroo',         # 18
    'laptop'            # 19
]

# LOAD TRAINING AND VALIDATION SETS

data_gen = ImageDataGenerator(rotation_range = 10,
                              width_shift_range = 10,
                              height_shift_range = 10,
                              zoom_range = 0.3,
                              horizontal_flip = True,
                              vertical_flip = True,
                              rescale = 1./255, 
                              validation_split = validation_split)

training_dir = os.path.join(cwd, "Classification_Dataset", "training")

print("Training")

training_generator = data_gen.flow_from_directory(training_dir,
                                        batch_size = bs,
                                        classes = classes,
                                        class_mode = 'categorical',
                                        shuffle = True,
                                        seed = SEED,
                                        subset = 'training')

print("\nValidation")

validation_generator = data_gen.flow_from_directory(training_dir,
                                        batch_size = bs,
                                        classes = classes,
                                        class_mode = 'categorical',
                                        shuffle = True,
                                        seed = SEED,
                                        subset = 'validation')

training_dataset = tf.data.Dataset.from_generator(lambda: training_generator,
                                              output_types = (tf.float32, tf.float32),
                                              output_shapes = ([None, img_w, img_h, 3], [None, len(classes)]))
validation_dataset = tf.data.Dataset.from_generator(lambda: validation_generator,
                                              output_types = (tf.float32, tf.float32),
                                              output_shapes = ([None, img_w, img_h, 3], [None, len(classes)]))

training_dataset = training_dataset.repeat()
validation_dataset = validation_dataset.repeat()

# WRITE FILENAMES TO JSON FILE

filenames = {
    "training" : {},
    "validation" : {}
}

for c in classes:
    filenames["training"][c] = []
    filenames["validation"][c] = []
    
    for fn in training_generator.filenames:
        if c in fn:
            filenames["training"][c].append(fn.replace(c + "/", ""))
    
    for fn in validation_generator.filenames:
        if c in fn:
            filenames["validation"][c].append(fn.replace(c + "/", ""))

with open('dataset_split.json', 'w') as file:
    json.dump(filenames, file, indent=4)
    

# LOAD TEST SET filenames

print("\nTo predict")

test_dir = os.path.join(cwd, "Classification_Dataset", "test")
test_filenames = next(os.walk(test_dir))[2]
test_filenames = list(filter(lambda fn: fn[-4:] == '.jpg', test_filenames))

print("Found " + str(len(test_filenames)) + " images.")

Training
Found 1406 images belonging to 20 classes.

Validation
Found 148 images belonging to 20 classes.

To predict
Found 500 images.


## Model building, fitting and predicting functions

In [4]:
def build_model(input_shape, conv_depth, start_num_filters, kernel_size, pool_size, fc_units, num_classes, kernel_regularizer = None):
    
    model = Sequential()
    
    for i in range(conv_depth):
        if i == 0:
            model.add(Conv2D(filters = start_num_filters,
                             kernel_size = kernel_size,
                             strides = (1, 1),
                             padding = 'same',
                             activation = 'relu',
                             input_shape = input_shape))
        else:
            model.add(Conv2D(filters = start_num_filters,
                             kernel_size = kernel_size,
                             strides = (1, 1),
                             padding = 'same',
                             activation = 'relu'))
            
        model.add(MaxPool2D(pool_size = pool_size))
        start_num_filters *= 2
        
    model.add(Flatten())
    
    for i in range(len(fc_units)):
        model.add(Dense(units = fc_units[i], 
                        activation = 'relu', 
                        kernel_regularizer = kernel_regularizer))
    
    model.add(Dense(units = num_classes, activation = 'softmax'))
    
    return model
    

In [5]:
def create_csv(results, results_dir='./'):

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')

In [6]:
def fit_model(model, model_name = datetime.now().strftime('%b%d_%H-%M-%S')):
    cwd = os.getcwd()
    
    # General experiments folder
    exps_dir = os.path.join(cwd, 'classification_experiments')
    if not os.path.exists(exps_dir):
        os.makedirs(exps_dir)
    
    now = datetime.now().strftime('%b%d_%H-%M-%S')
    
    # This experiment folder
    exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
    if not os.path.exists(exp_dir):
        os.makedirs(exp_dir)
    
    # Checpoints folder
    ckpt_dir = os.path.join(exp_dir, 'ckpts')
    if not os.path.exists(ckpt_dir):
        os.makedirs(ckpt_dir)
    
    # Checkpoints callback, best one will be the last saved
    ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp_{epoch:02d}.ckpt'), 
                                                       save_weights_only=True,
                                                       save_best_only=True, 
                                                       monitor='val_accuracy', 
                                                       mode = 'max')
    
    # Tensorboard folder
    tb_dir = os.path.join(exp_dir, 'tb_logs')
    if not os.path.exists(tb_dir):
        os.makedirs(tb_dir)
    
    # Tensorboard callback
    tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                                 profile_batch=0,
                                                 histogram_freq=1)  # if 1 shows weights histograms
    
    # Early stopping callback
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', 
                                                   patience=10,
                                                   mode = 'max')
    
    callbacks= [ckpt_callback, tb_callback, es_callback]
    
    model.fit(x=training_dataset,
              epochs=100,
              steps_per_epoch=len(training_generator),
              validation_data=validation_dataset,
              validation_steps=len(validation_generator), 
              callbacks=callbacks)
    
    # Load best model (last one saved)
    latest = tf.train.latest_checkpoint(ckpt_dir)
    print("Latest model: " + latest)
    model.load_weights(os.path.join(ckpt_dir, latest))
    
    return (model, exp_dir)

In [7]:
def predict(model, exp_dir):
    
    results = {}

    for image_name in test_filenames:
        img = Image.open(os.path.join(test_dir,image_name)).convert('RGB').resize((img_w, img_h))
        img_array = np.array(img)
        img_array = np.expand_dims(img_array, 0)
        img_array = np.divide(img_array,255)
        tensor = tf.convert_to_tensor(img_array, dtype = tf.float32)
        
        prediction = np.argmax(model.predict(tensor))
        results[image_name] = prediction

    create_csv(results = results, results_dir=exp_dir)

# Models

CNN seen in class

In [8]:
model_name = "CNN_1"
conv_depth = 5
start_num_filters = 8
kernel_size = (3, 3)
pool_size = (2, 2)
fc_units = [512]
num_classes = len(classes)

lr = 1e-3
optimizer = tf.keras.optimizers.Adam(learning_rate = lr)
loss = tf.keras.losses.CategoricalCrossentropy()
metrics = ['accuracy']

model = build_model(input_shape = (img_w, img_h, 3),
                   conv_depth = conv_depth,
                   start_num_filters = start_num_filters,
                   kernel_size = kernel_size,
                   pool_size = pool_size,
                   fc_units = fc_units,
                   num_classes = num_classes)

model.compile(optimizer = optimizer,
             loss = loss,
             metrics = metrics)

model.summary()

(model, exp_dir) = fit_model(model = model, model_name = model_name)

predict(model = model, exp_dir = exp_dir)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 256, 256, 8)       224       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 128, 8)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 16)      1168      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 32)        4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 64)        1

CNN seen in class + regularization

In [9]:
model_name = "CNN_2"
conv_depth = 5
start_num_filters = 8
kernel_size = (3, 3)
pool_size = (2, 2)
fc_units = [512]
num_classes = len(classes)

alpha = 0.01
kernel_regularizer = tf.keras.regularizers.l2(alpha)

lr = 1e-3
optimizer = tf.keras.optimizers.Adam(learning_rate = lr)
loss = tf.keras.losses.CategoricalCrossentropy()
metrics = ['accuracy']

model = build_model(input_shape = (img_w, img_h, 3),
                   conv_depth = conv_depth,
                   start_num_filters = start_num_filters,
                   kernel_size = kernel_size,
                   pool_size = pool_size,
                   fc_units = fc_units, 
                   kernel_regularizer = kernel_regularizer,
                   num_classes = num_classes)

model.compile(optimizer = optimizer,
             loss = loss,
             metrics = metrics)

model.summary()

(model, exp_dir) = fit_model(model = model, model_name = model_name)

predict(model = model, exp_dir = exp_dir)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 256, 256, 8)       224       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 128, 128, 8)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 128, 128, 16)      1168      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 64, 64, 16)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 64, 64, 32)        4640      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 32, 32, 64)       